<a href="https://colab.research.google.com/github/razzlestorm/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/Unit_3_Sprint_2_Homework_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import sqlite3

!wget https://github.com/razzlestorm/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

--2019-11-16 00:56:03--  https://github.com/razzlestorm/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/razzlestorm/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-11-16 00:56:04--  https://github.com/razzlestorm/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/razzlestorm/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-11-16 00:56:04--  https://raw.githubusercontent.com/razzlestorm/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introdu

In [0]:
conn = sqlite3.connect('/content/rpg_db.sqlite3?raw=true')
cur = conn.cursor()

def print_data(query):
    cur.execute(query)
    return cur.fetchall()[0][0]

In [55]:
# How many total Characters are there?
query = """
SELECT COUNT(character_id)
FROM charactercreator_character
"""
print_data(query)

302

In [62]:
# How many of each specific subclass?

class_list = ['charactercreator_cleric', 'charactercreator_fighter', 'charactercreator_mage', 'charactercreator_necromancer', 'charactercreator_thief']

def sql_table_count_loop(li):
    for xx in li:
        query = """
        SELECT COUNT(*)
        FROM {}
        """.format(xx)
        cur.execute(query)
        print(f'The {xx[17:]} class has {cur.fetchall()[0][0]} rows.')

sql_table_count_loop(class_list)


The cleric class has 75 rows.
The fighter class has 68 rows.
The mage class has 108 rows.
The necromancer class has 11 rows.
The thief class has 51 rows.


In [58]:
# How many total Items?

item_count = """
SELECT COUNT(*)
FROM armory_item
"""
print_data(item_count)

174

In [59]:
# How many of the Items are weapons? How many are not?

weapon_count = """
SELECT COUNT(item_id)
FROM armory_item
WHERE item_id IN (SELECT item_ptr_id FROM armory_weapon aw)
"""

non_weapon_count = """
SELECT COUNT(item_id)
FROM armory_item
WHERE item_id NOT IN (SELECT item_ptr_id FROM armory_weapon aw)
"""

print(print_data(weapon_count))
print(print_data(non_weapon_count))

37
137


In [73]:
# How many Items does each character have? (Return first 20 rows)


character_item_count = """
SELECT character_id, COUNT(item_id)
FROM charactercreator_character_inventory
WHERE item_id NOT IN (SELECT item_ptr_id FROM armory_weapon aw)
GROUP BY character_id
LIMIT 20
"""

cur.execute(character_item_count)
cur.fetchall()



[(1, 3),
 (2, 3),
 (3, 2),
 (4, 4),
 (5, 2),
 (6, 1),
 (7, 4),
 (8, 3),
 (9, 4),
 (10, 4),
 (11, 2),
 (12, 3),
 (13, 4),
 (14, 4),
 (15, 4),
 (16, 1),
 (17, 5),
 (18, 5),
 (19, 3),
 (21, 4)]

In [74]:
# How many Weapons does each character have? (Return first 20 rows)

character_weapon_count = """
SELECT character_id, COUNT(item_id)
FROM charactercreator_character_inventory
WHERE item_id IN (SELECT item_ptr_id FROM armory_weapon aw)
GROUP BY character_id
LIMIT 20
"""

cur.execute(character_weapon_count)
cur.fetchall()



[(5, 2),
 (7, 1),
 (11, 1),
 (20, 1),
 (22, 1),
 (23, 1),
 (26, 1),
 (27, 3),
 (29, 2),
 (30, 1),
 (32, 1),
 (34, 1),
 (35, 2),
 (36, 3),
 (37, 2),
 (38, 2),
 (39, 2),
 (40, 1),
 (41, 1),
 (47, 1)]

In [75]:
# On average, how many Items does each Character have?
character_avg_items = """
SELECT AVG(counted_items)
FROM (
SELECT COUNT(item_id) as counted_items
FROM charactercreator_character_inventory
WHERE item_id NOT IN (SELECT item_ptr_id FROM armory_weapon aw)
GROUP BY character_id
)
"""

cur.execute(character_avg_items)
cur.fetchall()


[(2.43859649122807,)]

In [76]:
# On average, how many Weapons does each character have?
character_avg_weapons = """
SELECT AVG(counted_items)
FROM (
SELECT COUNT(item_id) as counted_items
FROM charactercreator_character_inventory
WHERE item_id IN (SELECT item_ptr_id FROM armory_weapon aw)
GROUP BY character_id
)
"""

cur.execute(character_avg_weapons)
cur.fetchall()

[(1.3096774193548386,)]

In [88]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00476/buddymove_holidayiq.csv')
df.head()

,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
0,User 1,2,77,79,69,68,95
1,User 2,2,62,76,76,69,68
2,User 3,2,50,97,87,50,75
3,User 4,2,68,77,95,76,61
4,User 5,2,98,54,59,95,86


In [80]:
conn_new = sqlite3.connect('buddymove_holidayiq.sqlite3')

df2 = df.to_sql('my_table', conn_new)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [0]:
newcur = conn_new.cursor()

In [94]:
newcur.execute('SELECT * FROM my_table')
print(newcur.fetchall()[0][6]) # 4= Nature, 6= Shopping

68


In [111]:
row_query = """
SELECT COUNT(*)
FROM
(SELECT *
FROM my_table
WHERE Nature >= 100)
WHERE Shopping >= 100
"""

newcur.execute(row_query)
newcur.fetchall()

[(78,)]